# Creación de paquetes HTTP

### Descripción: 
Con este código se va a intentar crear diferentes tipos de paquetes HTTP, mediante IA generativa. Se pretende pasar una descripción de estos paquetes al modelo y que este sea capaz de crear todos los campos necesarios para replicar un paquete con esas mismas especificaciones.

In [ ]:
import random
import re
import pickle

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, TCP
from scapy.layers.http import *

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Step 1: Generación de la descipción de un paquete HTTP
---

### 1.1 Explicación del protocolo
<center><img src="https://www.oreilly.com/api/v2/epubs/1565925092/files/httpatomoreillycomsourceoreillyimages96840.png">
    
<center> Esta imagen es genérica para una conversación HTTP.  

    ***
El protocolo HTTP nos permite solicitar y recibir recursos de una fuente, es un protocolo cliente-servidor. HTTP es un protocolo sin estado, es decir: **no guarda ningún dato entre dos peticiones en la mísma sesión**. Un paquete HTTP se manda sobre TCP, por lo que, primero será necesario establecer la conversación TCP. Esto no será necesario en este script, dado que se crea en otro específico para conversaciones TCP. Digamos que este notebook serviría para crear los paquetes intermedios de la conversación establecida en ese otro notebook.

### 1.2 Creación de la query

In [2]:
query = "In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.\n\n"
query += "HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.\n"

query += "This is a HTTP request summary:\n"
query += "192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 \n"
query += "This is the command to generate the packet:\n"
query += 'IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net/", Path="/") \n\n'

query += "This is a HTTP response summary:\n"
query += "212.144.254.123 192.168.110.10 HTTP 166 HTTP/1.1 200 OK (text/html) \n"
query += "This is the command to generate the packet:\n"
query += 'IP(src="212.144.254.123",  dst="192.168.110.10") / TCP(sport=80, dport=4321, flags="A") /HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html" ) / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"'
query += "\n\n"
query += "Take note that this response is just an example, sent info might be different, you mus also take into account different types of responses depending on the response status code, like: 400 = Bad Request, 403= Forbidden, etc.\n"
query += "You may be asked to generate different types of HTTP packets than these ones, please take a few seconds to think and figure out the best way to generate each of the packets.\n\n"

query += "Pay special attention to the TCP ports, HTTP requests have port 80(http) as its destination port, while the sender port must be an ephemeral port(between 1025 and 655325) in http requests.\n"
query += "In the opposite direction, http responses have port 80 (http) as its sender port (sport), while having an ephimeral destination port(dport)\n"
query += "This is extremely important, so please, create a sport(for request) and dport(for responses) that make sense, they must me ephemeral ports and use them wuile creating the commands. You must follow the port structure already defined, dport=80 for http requests and sport=80 for http responses, create just the other side of the communication.\n"

In [3]:
print(query)

In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net/", Path="/") 

This is a HTTP response summary:
212.144.254.123 192.168.110.10 HTTP 166 HTTP/1.1 200 OK (text/html) 
This is the command to gen

## Step 2: Carga de posibles descripciones
---

In [5]:
with open("../../data/text/HTTP_TheUltimatePCAP_v2.txt","r") as f:
    packets_summary = f.read().splitlines()
print("Number of summaries: " + str(len(packets_summary)))

Number of summaries: 999


### 2.1 Carga de posibles conversaciones anteriores para concatenarlas

In [6]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [7]:
try:
    responses = pickle.load( open("../../data/Packets/HTTP/pickle/HTTP_packet_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []

## Step 3: Generación de comandos de paquetes conversaciones TCP
---

### 3.1 Randomización de resumenes a generar

In [9]:
## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
summaries = random.choices(packets_summary,k=200)
i=0

### 3.2 Solicitud al modelo para generación

In [15]:
#import math
summaries_per_request = 10

for i in range(int(len(packets_summary)/summaries_per_request)):
    
    query1 = query
    summaries = random.choices(packets_summary,k=summaries_per_request)

    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    

    text_sum = ""
    
    query1 += "\nBelow, I'll give you the descriptions of the packets you have to generate, the previous where just examples\n\n"
    
    for summary in summaries:
        query1 += "This is a packet summary:\n"
        query1 += summary + "\n"
        text_sum += summary + "\n"
        
    query1 += "\nGenerate python code for creating JUST the packets of the summaries given with scapy framework and put them all in a list named pkt_list. DO NOT create commands for other types of HTTP rather than the ones belonging to the packets wanted to generate\n"
    
    print(query1)
    
    #Esta es la petición que le mandas al modelo de IA
    completion = openai.completions.create(
        model="gpt-3.5-turbo-instruct", 
        prompt=query1,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    exec(responses[-1].completion.choices[0].text)
    
    with open("../../data/Packets/HTTP/pcap/HTTP2generated_3_5_turbo-instruct_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Packets/HTTP/pickle/HTTP2_packet_pairs_Aday_v3.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net/", Path="/") 

This is a HTTP respons

...Generated!
Number of completitions done: 23
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 4)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webe

...Generated!
Number of completitions done: 26
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 7)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webe

...Generated!
Number of completitions done: 29
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 10)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.web

...Generated!
Number of completitions done: 32
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 13)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.web

...Generated!
Number of completitions done: 35
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 16)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.web

...Generated!
Number of completitions done: 38
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 19)
In the next lines, I'll provide you with some packet descriptions for different types of HTTP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

HTTP packets can have various types and implement different methods, I will now try to explain the most important ones, using scapy 2.4.3.
This is a HTTP request summary:
192.168.110.10 212.144.254.123 HTTP 388 GET http://ip.webernetz.net/ HTTP/1.1 
This is the command to generate the packet:
IP(src="192.168.110.10",  dst="212.144.254.123") / TCP(sport=5678, dport=80) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.web

AttributeError: Port

### 3.3 Revisión de comandos generados

In [11]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

216.58.212.110 172.217.16.142        HTTP     165    HTTP/1.1 200 OK  (text/html)
77.88.99.110  59.60.70.80     HTTP   365    GET / HTTP/1.1
52.16.12.41   104.27.142.17          HTTP     370    GET /support HTTP/1.1
91.75.42.68   48.36.59.77     HTTP   155    HEAD / HTTP/1.0
67.45.89.12    56.78.91.23    HTTP   180    UNSUBSCRIBE /updates HTTP/1.1
172.67.228.11     198.41.206.141       HTTP     410    GET /news HTTP/1.1
198.41.206.140 45.63.123.89          HTTP     130    HEAD / HTTP/1.0
81.23.45.67    90.12.34.56    HTTP   200    HTTP/1.1 200 OK
104.28.12.40  91.198.174.198         HTTP     375    GET /pricing HTTP/1.1
185.60.216.35      198.51.100.17        HTTP     215    SUBSCRIBE /music/evt HTTP/1.1


pkt_list = [
    IP(src="216.58.212.110", dst="172.217.16.142") / TCP(sport=1234, dport=80) / HTTP() / HTTPResponse(Http_Version="HTTP/1.1", Status_Code="200", Reason_Phrase="OK", Content_Type="text/html"),
    IP(src="77.88.99.110", dst="59.60.70.80") / TCP(sport=2345, dport=80) / H

## Step 4: Generación de paquetes
---

In [212]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
#exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=tcp src=194.247.5.7 dst=85.25.246.38 |<TCP  sport=1234 dport=http |<HTTP  |<HTTPRequest  Method='GET' Path='/' Http_Version='HTTP/1.1' Host='netsec.blog/' |>>>>, <IP  frag=0 proto=tcp src=85.25.246.38 dst=194.247.5.7 |<TCP  sport=http dport=4321 flags=A |<HTTP  |<HTTPResponse  Http_Version='HTTP/1.1' Status_Code='301' Reason_Phrase='Moved Permanently' Content_Type='text/html' |<Raw  load='<HTML><BODY><H1>Redirecting to new page...</H1></BODY></HTML>' |>>>>>, <IP  frag=0 proto=tcp src=194.247.5.7 dst=85.25.246.38 |<TCP  sport=rrac dport=http |<HTTP  |<HTTPRequest  Method='CONNECT' Http_Version='HTTP/1.1' Host='weberblog.net/' |>>>>, <IP  frag=0 proto=tcp src=85.25.246.38 dst=194.247.5.7 |<TCP  sport=http dport=4321 flags=A |<HTTP  |<HTTPResponse  Http_Version='HTTP/1.1' Status_Code='200' Reason_Phrase='Connection established' |>>>>, <IP  frag=0 proto=tcp src=192.168.173.5 dst=194.247.5.23 |<TCP  sport=1234 dport=http |<HTTP  |<HTTPRequest  Method='GET' Path='/' Http_V

In [194]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 5 packets.


## Step 5: Introducción en un pcap
---

In [216]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Packets/HTTP/pcap/HTTP2generated_3_5_turbo-instruct_conv_ALL.pcap", "ba+") as f:
    wrpcap(f, pkt1, append=True)